# 05: Logikk

**Forfatter:** Benedikt Goodman \
**Medhjelpere:** Mistral Large, ChatGPT-4

Vi har nå vært innom mange av konseptene som er nødvendige å forstå dersom man skal sette seg inn i hva som foregår i programmer som skrives i Python. Utover datatyper, funksjoner og klasser er logikk en svært viktig byggeklosse å skrive gode programmer.

## Logic in Python

### Boolean Values and Expressions

In Python, a boolean value is either True or False. We can create boolean expressions using comparison operators and logical operators. One of the simplest ways is to use the comparison operators.

Comparison operators allow us to compare two values and return a boolean value. The most common comparison operators are `==` (equal to), `!=` (not equal to), `>` (greater than), `<` (less than), `>=` (greater than or equal to), and `<=` (less than or equal to).

In [1]:
# Creates a True boolean via larger than comparison
print(type(2 > 1))

# not equal to comparison
print(2 != 2)

#equal or larger than
print(2 >= 2)

# Equal or smaller than
print(2 <= 1)

# equal to comparison
print(True == 'string')

<class 'bool'>
False
True
False
False


The `is` operator is used to check whether two variables refer to the same object in memory, rather than having the same value. This is in contrast to the `==` operator, which checks if the values of two variables are the same. Both will however yield a boolean value and can thus be used as logical operators.

In [2]:
x = True
x is True

True

In [3]:
x is False

False

In [4]:
a = [1, 2, 3]
b = a
c = [1, 2, 3]

# True, because both a and b point to the same list
print(a is b)

# False, because c is a different list object, despite having the same content
print(a is c)

True
False


In [5]:
# What will happen here?
b = a.copy()

print(b == a)

True


To produce booleans we can make functions which do evaluations and logic and then return boolean values

In [6]:
def is_even(n):
    # Will evaluate to True if we use an even number 
    # all even numbers have a modulus of 2 equal to zero
    if n % 2 == 0:
        return True
    else:
        return False

### Inbuilt functions which return booleans



In Python, some built-in functions return boolean values, which can be used for logic and control flow in your code. One such function is `isinstance()`, which you will see used a lot for different types of validation.

The `isinstance()` function takes two arguments: an `object` and a `class` or `tuple` of classes, and returns `True` if the `object` is an instance of the specified class(es), and `False` otherwise.

In [7]:
isinstance('i am a string', str)

True

Normally we use this kind of function as a guard-clause where we usually check if something is *not* the case. Why? Because it makes for code that is less convoluted has less indentations and will always have the so-called "happy-path" at the bottom. This makes for code that is easily readable.

In [8]:
# Good use of isinstance
def square_number(x: int | float):
    # We use isinstance as a "type-guard"
    if not isinstance(x, (int, float)):
        raise TypeError("x must be int or float")

    return x**x


def square_number_bad(x: int | float):
    # Isinstance is used to provide happy path
    if isinstance(x, (int, float)):
        return x**x
    else:
        raise TypeError("x must be int or float")



### Conditional statements

Conditional statements allow us to execute different blocks of code depending on whether a boolean expression is `True` or `False`. This allows us to tell Python which codeblocks to run. The most common conditional statements are `if`, `if-else`, and `if-elif-else`.

In [9]:
def get_grade(score, other_value):
    if score >= 90:
        return "A"
    elif score >= 80:
        return "B"
    elif score >= 70:
        return "C"
    elif score >= 60:
        return "D"
    else:
        return "F"

One can also use `case` to achieve the same logic if using python 3.10 and above. I personally recommend using the if-elif-else patterns though.


In [10]:
def get_grade_case(score):
    match divmod(score, 10)[0]:
        case 9 | 10:
            return "A"
        case 8:
            return "B"
        case 7:
            return "C"
        case 6:
            return "D"
        case _:
            return "F"

## Logic in objects - with a brief note on properties

If we add logic to our objects, we can quite quickly create complex and responsive behaviour. Lets first make some objects to play around with.

In [11]:
from pydantic import validate_call


class Wine:
    @validate_call
    def __init__(self, winetype: str):
        # By putting a _ at the start of a variable name we hide the variable from the linter -> it becomes a "private variable"
        self._winetype = winetype
    
    # This is a getter-method, an attribute which only has a getter-method is considered "read-only"
    @property
    def winetype(self):
        """Returns lowercased version of whatever self._grape is"""
        return self._winetype.lower()
    
    # This is a setter method, if an attribute has this it becomes re-assignable
    @winetype.setter
    @validate_call
    def winetype(self, new_winetype: str):
        self._winetype = new_winetype
    
# make the object
chianti = Wine('Chianti Classico')

# the property wrapper allows us to access a lowecased version self._grape by writing obj.grape
chianti.winetype

# It also prevents users from modifying the attribute after the object has been created
# Make some wine objects
# chianti = Wine('Chianti Classico')
# shitty_chenin = Wine('Shitty Chenin Blanc')

'chianti classico'

### A quick note on @property wrappers

In Python, `@property` wrappers are used to control access to the `attributes` of a class, allowing you to define methods that are accessed like attributes. This is particularly useful for encapsulating behavior, validating data, or implementing computed properties without altering the external interface of your class. The property decorator provides a built-in way to achieve this by defining getter, setter, and deleter functions for a class attribute.
Usage of Property Wrappers

**Getter Method**: The getter method is used to access the value of a property. It is defined first and decorated with `@property`. This method returns the internal state of a private attribute.

**Setter Method**: The setter method is used to set or update the value of a property. It is named the same as the getter method but is decorated with `@property_name.setter`, where property_name is the name of the property. This method often includes validation logic.

In [12]:
# Will not work, unless we activate the setter method
chianti.winetype = 'Cabernet Frank'

Now lets make an object which uses the Wine class and applies logic that depends on the state of the wine-objects.

In [13]:
class Benny:
    """A picky object which does not like undesirable traits in its received wines"""
    def __init__(self, undesirable_traits: set[str], wine: Wine):
        
        # instance variables
        self._undesirable_traits = undesirable_traits
        self._wine = wine
        
        # Benny checks the wine upon receiving it. Will be True or False.
        self._benny_likes_it = self.check_wine()

    def check_wine(self):
        """Returns False if received wine overlaps with undesirable traits, returns True otherwise"""
        wine_type = set(self._wine.lower().split())

        if self._undesirable_traits.intersection(wine_type):
            return False
        
        else:
            return True

    def taste_wine(self):
        """Returns output based on status of self._benny_likes_it variable"""
        if self._benny_likes_it:
            print(f"Benny says: Oh yes, {self._wine}, thats proper lovely, that.")
            
        else:
            print(
                f"Benny frowns: Ugh, {self._wine}, not quite my taste."
            )


In [14]:
# Make some wine objects
chianti = Wine('Chianti Classico')
shitty_chenin = Wine('Shitty Chenin Blanc')

undesirable_traits = {'cheap', 'shitty'}

# Make some instances of benny with wine
benny_with_chianti = Benny(undesirable_traits, chianti.winetype)
benny_with_shitty_chenin = Benny(undesirable_traits, shitty_chenin.winetype)

In [15]:
benny_with_chianti.taste_wine()

Benny says: Oh yes, chianti classico, thats proper lovely, that.


In [16]:
benny_with_shitty_chenin.taste_wine()

Benny frowns: Ugh, shitty chenin blanc, not quite my taste.


### Ok great, when is this useful?

Logic within objects is used to manage and manipulate the state of an object based on the values of its attributes. Using logic in objects can make your code more expressive and powerful, since it allows you to define complex behavior for your objects based on their attributes and other factors. It also allows you to encapsulate this behavior within your objects, which can make your code more modular and easier to maintain. Which is generally what you're aiming for.

Here are some use-cases for applying logic in relation to classes:
Initialization and State Management: Constructors initialize object states with validation and setup.
Encapsulation: Methods encapsulate complex behaviors, ensuring states remain valid and operations are restricted to maintain integrity.
Decision Making: Conditional logic within methods allows objects to perform different actions based on their state or inputs.
Polymorphism and Method Overriding: Logic in methods can be customized in subclasses to alter or enhance functionality.
Utility Methods: Objects often include methods that provide specific functionality, helping to organize code and increase reusability.

### Logical Operators

Logical operators allow us to combine boolean expressions using `and`, `or`, and `not`. These can be used whenever we are evaluation Boolean values.

In [17]:
def more_complex_logic(x: int | float):
    if not isinstance(x, (int, float)):
        raise TypeError('Blæ')
    
    # X is larger or equal to 10 and an even number
    if x >= 10 and x % 2 == 0:
        return x ** 2
    
    # X is smaller than 10 or an odd number
    elif x < 10 or x % 2 != 0:
        return x ** -2
    
    elif x == 42:
        print('Thats the meaning of life, man.')
    
    # X is smaller than 10 but an even number
    else:
        return x * 2

### Truthy and Falsy Values

In Python, certain values are considered "truthy" or "falsy" when used in a boolean context. For example, 0, None, and empty strings are considered falsy, while non-zero numbers, non-empty strings, and objects are considered truthy. Here are some built-in 

Here are some examples of falsy values in Python:

- `None`
- `False`
- `0` (zero)
- `0.0` (zero as a float)
- `""` (empty string)
- `[]` (empty list)
- `()` (empty tuple)
- `{}` (empty dictionary)

Here are some truthy objects in Python:
- `True`
- `1` (one)
- `3.14` (pi as a float)
- `"hello"` (non-empty string)
- `[1, 2, 3]` (non-empty list)
- `(1, 2, 3)` (non-empty tuple)
- `{"a": 1, "b": 2}` (non-empty dictionary)


In [18]:
# proof
x = 0

if x:
    print("x is truthy")
else:
    print("x is falsy")
    
name = ""

# Normally we write these truthy and falsy values like so because programmers are lazy
if not name:
    print("Hello, Anonymous!")

name = 'Steve'

if name:
    print(f"Hello, {name}")

x is falsy
Hello, Anonymous!
Hello, Steve


In [19]:
# Example with list
l = []

if not l:
    print('l is empty falsy')
    
l.append(':D')

if l:
    print(f'l contains {l} and is truthy')

l is empty falsy
l contains [':D'] and is truthy


### Chaining Comparisons

Multiple comparison operators in a single expression to create a `bool` value.

In [20]:
def is_in_range(n, start, end):
    # The two comparisons we do here will evaluate to a singular True or False value
    return start <= n <= end

is_in_range(10, 5, 15)

True

### try-except blocks
 
`try` and `except` blocks are used for error handling. This is also known as exception handling. They allow you to catch and handle errors that may occur during the execution of your code, so that your program can continue running gracefully instead of crashing. They kind of work like logic, which is why they are apart of this lecture.

Here's the basic syntax of a try and except block:

In [21]:
try:
    x = 1 / 0
except ZeroDivisionError:
    print("Cannot divide by zero")


Cannot divide by zero


We normally specify all types of exceptions we want to catch with the except terms. The keyword finally trigger stuff regardless of what comes before if we use try and except.

In [22]:
try:
    x = int(input("Enter a number: "))
    y = 1 / x
    print(y)
except ValueError:
    print("Invalid input")
except ZeroDivisionError:
    print("Cannot divide by zero")
finally:
    print("Thank you for using the program")


Enter a number:  3


0.3333333333333333
Thank you for using the program


Now that you know about try and except, here is a note on how to *not* use it. The thing you want to avoid is using a bare try and except block. While the code will run the except block if the try block fails, it will do so for any reason. I.e. any bug will trigger this behaviour and it will be unclear why the program behaves like it does.

In [23]:
try:
    x = int(input("Enter a number: "))
    y = 1 / x
    print(y)
# will trigger if the above fails for whatever reason
# If you write a bare except block like this into your programs
# they will be increadibly unstable
# DO NOT DO THIS
except:
    print('In a more complex program it would be very hard to figure out why this bare except block triggered.')

Enter a number:  4


0.25
